In [1]:
!pip install ultralytics

import cv2
import os
import dlib
import torch
import torch.nn as nn
import subprocess
import numpy as np
from sklearn.preprocessing import LabelEncoder
from ultralytics import YOLO
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall

In [2]:
device_str = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO("yolo11n.pt").to(device_str)
model.model = model.model.eval().to(device_str)
model_face = YOLO("/kaggle/input/yolo-face-model/yolov11n-face.pt").to(device_str)
predictor = dlib.shape_predictor('/kaggle/input/dlib-68-face-predictor/shape_predictor_68_face_landmarks.dat')

---

# Feature Extraction

In [4]:
def get_rotation(video_path):
    try:
        result = subprocess.run([
            'ffprobe', '-v', 'error',
            '-select_streams', 'v:0',
            '-show_entries', 'stream_tags=rotate',
            '-of', 'default=nokey=1:noprint_wrappers=1',
            video_path
        ], capture_output=True, text=True)
        return int(result.stdout.strip())
    except:
        return 0

def rotate_frame(frame, degrees):
    if degrees == 90:
        return cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    elif degrees == 180:
        return cv2.rotate(frame, cv2.ROTATE_180)
    elif degrees == 270:
        return cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return frame

In [5]:
def calculate_EAR(eye_points):
    A = np.linalg.norm(eye_points[1] - eye_points[5])
    B = np.linalg.norm(eye_points[2] - eye_points[4])
    C = np.linalg.norm(eye_points[0] - eye_points[3])
    ear = (A + B) / (2.0 * C)
    return ear


def calculate_MAR(mouth_points):
    A = np.linalg.norm(mouth_points[1] - mouth_points[7])
    B = np.linalg.norm(mouth_points[2] - mouth_points[6])
    C = np.linalg.norm(mouth_points[3] - mouth_points[5])
    D = np.linalg.norm(mouth_points[0] - mouth_points[4])
    mar = (A + B + C) / (2.0 * D)
    return mar


def calculate_HPE(landmarks, frame_shape):
    h, w = frame_shape[:2]

    image_points = np.array([
        landmarks[30],
        landmarks[8],
        landmarks[36],
        landmarks[45],
        landmarks[48],
        landmarks[54]
    ], dtype="double")

    model_points = np.array([
        (0.0, 0.0, 0.0),
        (0.0, -330.0, -65.0),
        (-225.0, 170.0, -135.0),
        (225.0, 170.0, -135.0),
        (-150.0, -150.0, -125.0),
        (150.0, -150.0, -125.0)
    ])

    # Camera internals
    focal_length = w
    center = (w / 2, h / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")

    dist_coeffs = np.zeros((4, 1))

    success, rotation_vector, translation_vector = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
    )

    if not success:
        return np.nan, np.nan, np.nan

    rotation_mat, _ = cv2.Rodrigues(rotation_vector)
    proj_mat = np.hstack((rotation_mat, translation_vector))
    _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(proj_mat)

    pitch, yaw, roll = euler_angles.flatten().astype(float)
    return pitch, yaw, roll

In [6]:
def get_yolo_features(frame):
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = torch.from_numpy(img).float().permute(2, 0, 1).unsqueeze(0).to(device_str) / 255.0
    with torch.no_grad():
        # Forward through YOLO backbone only
        backbone_out = model.model.model[:10](img)
        pooled = torch.nn.functional.adaptive_avg_pool2d(backbone_out[-1], (1, 1))
        feats = pooled.view(pooled.size(0), -1).squeeze(0).cpu().numpy()
        return feats

In [7]:
def get_features(current_data, current_num, total_num):
    features = []
    yolo_features = []
    vid = cv2.VideoCapture(current_data)
    rotation = get_rotation(current_data)
    num_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

    for _ in tqdm(range(num_frames), desc=f"{current_num}/{total_num}"):
        ret, frame = vid.read()
        if not ret or frame is None:
            continue
        frame = rotate_frame(frame, rotation)
        if frame is None:
            continue
        pred = model_face(frame, device=device_str, verbose=False)[0]
        if len(pred) != 0:
            for i, box in enumerate(pred.boxes):
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                rect = dlib.rectangle(left=x1, top=y1, right=x2, bottom=y2)
                face_landmark = predictor(frame, rect)
                break

                
            img = frame[y1:y2+1, x1:x2+1]
            yolo_features.append(get_yolo_features(img))
            
            landmarks = np.array([[face_landmark.part(i).x, face_landmark.part(i).y] for i in range(68)])
            
            left_eye = landmarks[36:42]
            right_eye = landmarks[42:48]
            ear = (calculate_EAR(left_eye) + calculate_EAR(right_eye)) / 2.0
            
            mouth = landmarks[60:68]
            mar = calculate_MAR(mouth)
            
            pitch, yaw, roll = calculate_HPE(landmarks, frame.shape)
            features.append([ear, mar, pitch, yaw, roll])
        else:
            features.append([np.nan]*5)
            yolo_features.append([np.nan]*256)
    return features, yolo_features

In [3]:
dataX = [[] for _ in range(5)]
dataY = [[] for _ in range(5)]


for dirname, _, filenames in os.walk('/kaggle/input/uta-reallife-drowsiness-dataset/Fold1_part1/Fold1_part1'):
    for filename in filenames:
        dataX[0].append(os.path.join(dirname, filename))
        dataY[0].append(filename.split(".")[0])

for dirname, _, filenames in os.walk('/kaggle/input/uta-reallife-drowsiness-dataset/Fold1_part2/Fold1_part2'):
    for filename in filenames:
        dataX[0].append(os.path.join(dirname, filename))
        dataY[0].append(filename.split(".")[0])


for dirname, _, filenames in os.walk('/kaggle/input/uta-reallife-drowsiness-dataset/Fold2_part1/Fold2_part1'):
    for filename in filenames:
        dataX[1].append(os.path.join(dirname, filename))
        dataY[1].append(filename.split(".")[0])

for dirname, _, filenames in os.walk('/kaggle/input/uta-reallife-drowsiness-dataset/Fold2_part2/Fold2_part2'):
    for filename in filenames:
        dataX[1].append(os.path.join(dirname, filename))
        dataY[1].append(filename.split(".")[0])


for dirname, _, filenames in os.walk('/kaggle/input/uta-reallife-drowsiness-dataset/Fold3_part1/Fold3_part1'):
    for filename in filenames:
        dataX[2].append(os.path.join(dirname, filename))
        dataY[2].append(filename.split(".")[0])

f3_spCase_x = []
for dirname, _, filenames in os.walk('/kaggle/input/uta-reallife-drowsiness-dataset/Fold3_part2/Fold3_part2'):
    for filename in filenames:
        if filename.split(".")[0] == "10_1":
            f3_spCase_x.append(os.path.join(dirname, filename))
        elif filename.split(".")[0] == "10_2":
            f3_spCase_x.append(os.path.join(dirname, filename))
            dataX[2].append(f3_spCase_x)
            dataY[2].append("10")
        else:
            dataX[2].append(os.path.join(dirname, filename))
            dataY[2].append(filename.split(".")[0])


for dirname, _, filenames in os.walk('/kaggle/input/uta-reallife-drowsiness-dataset/Fold4_part1/Fold4_part1'):
    for filename in filenames:
        dataX[3].append(os.path.join(dirname, filename))
        dataY[3].append(filename.split(".")[0])

for dirname, _, filenames in os.walk('/kaggle/input/uta-reallife-drowsiness-dataset/Fold4_part2/Fold4_part2'):
    for filename in filenames:
        dataX[3].append(os.path.join(dirname, filename))
        dataY[3].append(filename.split(".")[0])


f5_spCase_x = []
for dirname, _, filenames in os.walk('/kaggle/input/uta-rldd-fold5/Fold5_part1/Fold5_part1'):
    for filename in filenames:
        if filename.split(".")[0] == "10_1":
            f5_spCase_x.append(os.path.join(dirname, filename))
        elif filename.split(".")[0] == "10_2":
            f5_spCase_x.append(os.path.join(dirname, filename))
            dataX[4].append(f5_spCase_x)
            dataY[4].append("10")
        else:
            dataX[4].append(os.path.join(dirname, filename))
            dataY[4].append(filename.split(".")[0])

for dirname, _, filenames in os.walk('/kaggle/input/uta-rldd-fold5/Fold5_part2/Fold5_part2'):
    for filename in filenames:
        dataX[4].append(os.path.join(dirname, filename))
        dataY[4].append(filename.split(".")[0])



le = LabelEncoder().fit(["0", "5", "10"])
dataY = [le.transform(d) for d in dataY]


print(len(dataX), len(dataY), ":")
print(len(dataX[0]), len(dataY[0]))
print(len(dataX[1]), len(dataY[1]))
print(len(dataX[2]), len(dataY[2]))
print(len(dataX[3]), len(dataY[3]))
print(len(dataX[4]), len(dataY[4]))

5 5 :
36 36
36 36
36 36
36 36
36 36


In [27]:
i = 0
total_num = len(dataX[i])
labels = np.array(dataY[i])

all_features = []
all_yolo_features = []
lengths = []

for j in range(total_num):
    feats = []
    yolo_feats = []
    if isinstance(dataX[i][j], list):
        for k in range(len(dataX[i][j])):
            temp_feats, temp_yolo_feats = get_features(dataX[i][j][k], j+1, total_num)
            feats.extend(temp_feats)
            yolo_feats.extend(temp_yolo_feats)
    else:
        feats, yolo_feats = get_features(dataX[i][j], j+1, total_num)
    all_features.append(feats)
    all_yolo_features.append(yolo_feats)
    lengths.append(len(feats))

all_features = np.array(all_features, dtype="object")
all_yolo_features = np.array(all_yolo_features, dtype="object")
lengths=np.array(lengths)

np.savez(f"/kaggle/working/fold_{i+1}", features=all_features, yolo_features=all_yolo_features, lengths=lengths, labels=labels)

36/36: 100%|██████████| 18818/18818 [07:58<00:00, 39.30it/s]


In [8]:
i = 1
total_num = len(dataX[i])
labels = np.array(dataY[i])

all_features = []
all_yolo_features = []
lengths = []

for j in range(total_num):
    feats = []
    yolo_feats = []
    if isinstance(dataX[i][j], list):
        for k in range(len(dataX[i][j])):
            temp_feats, temp_yolo_feats = get_features(dataX[i][j][k], j+1, total_num)
            feats.extend(temp_feats)
            yolo_feats.extend(temp_yolo_feats)
    else:
        feats, yolo_feats = get_features(dataX[i][j], j+1, total_num)
    all_features.append(feats)
    all_yolo_features.append(yolo_feats)
    lengths.append(len(feats))

all_features = np.array(all_features, dtype="object")
all_yolo_features = np.array(all_yolo_features, dtype="object")
lengths=np.array(lengths)

np.savez(f"/kaggle/working/fold_{i+1}", features=all_features, yolo_features=all_yolo_features, lengths=lengths, labels=labels)

36/36: 100%|██████████| 11045/11045 [04:46<00:00, 38.58it/s]


In [9]:
i = 2
total_num = len(dataX[i])
labels = np.array(dataY[i])

all_features = []
all_yolo_features = []
lengths = []

for j in range(total_num):
    feats = []
    yolo_feats = []
    if isinstance(dataX[i][j], list):
        for k in range(len(dataX[i][j])):
            temp_feats, temp_yolo_feats = get_features(dataX[i][j][k], j+1, total_num)
            feats.extend(temp_feats)
            yolo_feats.extend(temp_yolo_feats)
    else:
        feats, yolo_feats = get_features(dataX[i][j], j+1, total_num)
    all_features.append(feats)
    all_yolo_features.append(yolo_feats)
    lengths.append(len(feats))

all_features = np.array(all_features, dtype="object")
all_yolo_features = np.array(all_yolo_features, dtype="object")
lengths=np.array(lengths)

np.savez(f"/kaggle/working/fold_{i+1}", features=all_features, yolo_features=all_yolo_features, lengths=lengths, labels=labels)

36/36: 100%|██████████| 17651/17651 [06:49<00:00, 43.05it/s]


In [8]:
i = 3
total_num = len(dataX[i])
labels = np.array(dataY[i])

all_features = []
all_yolo_features = []
lengths = []

for j in range(total_num):
    feats = []
    yolo_feats = []
    if isinstance(dataX[i][j], list):
        for k in range(len(dataX[i][j])):
            temp_feats, temp_yolo_feats = get_features(dataX[i][j][k], j+1, total_num)
            feats.extend(temp_feats)
            yolo_feats.extend(temp_yolo_feats)
    else:
        feats, yolo_feats = get_features(dataX[i][j], j+1, total_num)
    all_features.append(feats)
    all_yolo_features.append(yolo_feats)
    lengths.append(len(feats))

all_features = np.array(all_features, dtype="object")
all_yolo_features = np.array(all_yolo_features, dtype="object")
lengths=np.array(lengths)

np.savez(f"/kaggle/working/fold_{i+1}", features=all_features, yolo_features=all_yolo_features, lengths=lengths, labels=labels)

36/36: 100%|██████████| 18183/18183 [08:50<00:00, 34.29it/s]


In [8]:
i = 4
total_num = len(dataX[i])
labels = np.array(dataY[i])

all_features = []
all_yolo_features = []
lengths = []

for j in range(total_num):
    feats = []
    yolo_feats = []
    if isinstance(dataX[i][j], list):
        for k in range(len(dataX[i][j])):
            temp_feats, temp_yolo_feats = get_features(dataX[i][j][k], j+1, total_num)
            feats.extend(temp_feats)
            yolo_feats.extend(temp_yolo_feats)
    else:
        feats, yolo_feats = get_features(dataX[i][j], j+1, total_num)
    all_features.append(feats)
    all_yolo_features.append(yolo_feats)
    lengths.append(len(feats))

all_features = np.array(all_features, dtype="object")
all_yolo_features = np.array(all_yolo_features, dtype="object")
lengths=np.array(lengths)

np.savez(f"/kaggle/working/fold_{i+1}", features=all_features, yolo_features=all_yolo_features, lengths=lengths, labels=labels)

36/36: 100%|██████████| 18220/18220 [09:28<00:00, 32.02it/s]
